In [27]:
!pip install datasets transformers[sentencepiece]
!pip install accelerate

In [28]:
from huggingface_hub import notebook_login

notebook_login()


Login successful
Your token has been saved to /root/.huggingface/token
Authenticated through git-credential store but this isn't the helper defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub. Run the following command in your terminal in case you want to set this credential helper as the default

git config --global credential.helper store


In [29]:

from datasets import load_dataset, load_metric

raw_datasets = load_dataset("json", data_files="small_data_DS_format.json")

Using custom data configuration default-8243a3620de5050a
Reusing dataset json (/root/.cache/huggingface/datasets/json/default-8243a3620de5050a/0.0.0/ac0ca5f5289a6cf108e706efcf040422dbbfa8e658dee6a819f20d76bb84d26b)


  0%|          | 0/1 [00:00<?, ?it/s]

In [30]:
from datasets import load_dataset, load_metric

raw_datasets = load_dataset("json", data_files="small_data_DS_format.json")
split_datasets = raw_datasets["train"].train_test_split(train_size=0.9, seed=20)
split_datasets["validation"] = split_datasets.pop("test")

from transformers import pipeline

model_checkpoint = "Helsinki-NLP/opus-mt-en-fr"
translator = pipeline("translation", model=model_checkpoint)

from transformers import AutoTokenizer

model_checkpoint = "Helsinki-NLP/opus-mt-en-fr"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, return_tensors="tf")

max_input_length = 23
max_target_length = 23


def preprocess_function(examples):
    inputs = [ex["en"] for ex in examples["translation"]]
    targets = [ex["fr"] for ex in examples["translation"]]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)

    # Set up the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=max_target_length, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized_datasets = split_datasets.map(
    preprocess_function,
    batched=True,
    remove_columns=split_datasets["train"].column_names,
)




Using custom data configuration default-8243a3620de5050a
Reusing dataset json (/root/.cache/huggingface/datasets/json/default-8243a3620de5050a/0.0.0/ac0ca5f5289a6cf108e706efcf040422dbbfa8e658dee6a819f20d76bb84d26b)


  0%|          | 0/1 [00:00<?, ?it/s]

Loading cached split indices for dataset at /root/.cache/huggingface/datasets/json/default-8243a3620de5050a/0.0.0/ac0ca5f5289a6cf108e706efcf040422dbbfa8e658dee6a819f20d76bb84d26b/cache-2cfd569c91da1a68.arrow and /root/.cache/huggingface/datasets/json/default-8243a3620de5050a/0.0.0/ac0ca5f5289a6cf108e706efcf040422dbbfa8e658dee6a819f20d76bb84d26b/cache-c68764012c4dc9d1.arrow
loading configuration file https://huggingface.co/Helsinki-NLP/opus-mt-en-fr/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/5ad88432037ab18b1eb95761258d2b1b3a32e1e401d5f610f86eb3f479e59e8c.2b4f07b3f8de3922d42e6312c55d0597e44d2273507e7c5d0b6daf75fb2cc673
Model config MarianConfig {
  "_name_or_path": "Helsinki-NLP/opus-mt-en-fr",
  "_num_labels": 3,
  "activation_dropout": 0.0,
  "activation_function": "swish",
  "add_bias_logits": false,
  "add_final_layer_norm": false,
  "architectures": [
    "MarianMTModel"
  ],
  "attention_dropout": 0.0,
  "bad_words_ids": [
    [
      59513
    ]


In [31]:
from transformers import AutoModelForSeq2SeqLM

model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

batch = data_collator([tokenized_datasets["train"][i] for i in range(1, 3)])

loading configuration file https://huggingface.co/Helsinki-NLP/opus-mt-en-fr/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/5ad88432037ab18b1eb95761258d2b1b3a32e1e401d5f610f86eb3f479e59e8c.2b4f07b3f8de3922d42e6312c55d0597e44d2273507e7c5d0b6daf75fb2cc673
Model config MarianConfig {
  "_name_or_path": "Helsinki-NLP/opus-mt-en-fr",
  "_num_labels": 3,
  "activation_dropout": 0.0,
  "activation_function": "swish",
  "add_bias_logits": false,
  "add_final_layer_norm": false,
  "architectures": [
    "MarianMTModel"
  ],
  "attention_dropout": 0.0,
  "bad_words_ids": [
    [
      59513
    ]
  ],
  "bos_token_id": 0,
  "classif_dropout": 0.0,
  "classifier_dropout": 0.0,
  "d_model": 512,
  "decoder_attention_heads": 8,
  "decoder_ffn_dim": 2048,
  "decoder_layerdrop": 0.0,
  "decoder_layers": 6,
  "decoder_start_token_id": 59513,
  "dropout": 0.1,
  "encoder_attention_heads": 8,
  "encoder_ffn_dim": 2048,
  "encoder_layerdrop": 0.0,
  "encoder_layers": 6,
  "e

In [32]:
!pip install sacrebleu

from datasets import load_metric

metric = load_metric("sacrebleu")

import numpy as np


def compute_metrics(eval_preds):
    preds, labels = eval_preds
    # In case the model returns more than the prediction logits
    if isinstance(preds, tuple):
        preds = preds[0]

    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    # Replace -100s in the labels as we can't decode them
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Some simple post-processing
    decoded_preds = [pred.strip() for pred in decoded_preds]
    decoded_labels = [[label.strip()] for label in decoded_labels]
    # print("Return:- ", metric.compute(predictions=decoded_preds, references=decoded_labels))
    # print("decoded_preds:- ", decoded_preds)
    # print("decoded_labels:- ", decoded_labels)
    # print("Done")
    return metric.compute(predictions=decoded_preds, references=decoded_labels)




In [ ]:
## earlier tried config

args = Seq2SeqTrainingArguments(
    f"marian-finetuned-kde4-en-to-fr",
    evaluation_strategy="no",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=64,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=1,
    predict_with_generate=True,
    fp16=True,
    push_to_hub=False,
    report_to = 'wandb',
    optim = 'adafactor' ## tried this optimiser
)

In [33]:
from transformers import Seq2SeqTrainingArguments

args = Seq2SeqTrainingArguments(
    f"marian-finetuned-kde4-en-to-fr",
    evaluation_strategy="no",
    save_strategy="epoch",
    learning_rate=0.01,  ## reduced learning rate
    per_device_train_batch_size=32,
    per_device_eval_batch_size=64,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=1,
    predict_with_generate=True,
    fp16=True,
    push_to_hub=False,
    report_to = 'wandb'
)


from transformers import Seq2SeqTrainer

trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

# trainer.evaluate(max_length=max_target_length)

PyTorch: setting up devices
Using amp half precision backend


In [34]:
trainer.train()

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use thePyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 124074
  Num Epochs = 1
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 3878
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"
wandb: Currently logged in as: parthbkgadoya (use `wandb login --relogin` to force relogin)


Step,Training Loss
500,6.443600


Step,Training Loss
500,6.443600
1000,4.505200
1500,4.494900
2000,4.481100
2500,4.472900
3000,0.899200
3500,0.000000


Saving model checkpoint to marian-finetuned-kde4-en-to-fr/checkpoint-3878
Configuration saved in marian-finetuned-kde4-en-to-fr/checkpoint-3878/config.json
Model weights saved in marian-finetuned-kde4-en-to-fr/checkpoint-3878/pytorch_model.bin
tokenizer config file saved in marian-finetuned-kde4-en-to-fr/checkpoint-3878/tokenizer_config.json
Special tokens file saved in marian-finetuned-kde4-en-to-fr/checkpoint-3878/special_tokens_map.json


Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=3878, training_loss=3.261575345712694, metrics={'train_runtime': 3865.9101, 'train_samples_per_second': 32.094, 'train_steps_per_second': 1.003, 'total_flos': 670206691639296.0, 'train_loss': 3.261575345712694, 'epoch': 1.0})

In [11]:
trainer.evaluate(max_length=max_target_length)

***** Running Evaluation *****
  Num examples = 13786
  Batch size = 64


Trainer is attempting to log a value of "[168147, 152538, 137483, 122891]" of type <class 'list'> for key "eval/counts" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "[170637, 156851, 143065, 129279]" of type <class 'list'> for key "eval/totals" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "[98.54076196838903, 97.2502566129639, 96.09827700695489, 95.05874890740182]" of type <class 'list'> for key "eval/precisions" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.


{'epoch': 1.0,
 'eval_bp': 1.0,
 'eval_counts': [168147, 152538, 137483, 122891],
 'eval_loss': 0.022329561412334442,
 'eval_precisions': [98.54076196838903,
  97.2502566129639,
  96.09827700695489,
  95.05874890740182],
 'eval_ref_len': 170441,
 'eval_runtime': 559.7025,
 'eval_samples_per_second': 24.631,
 'eval_score': 96.7283081766182,
 'eval_steps_per_second': 0.386,
 'eval_sys_len': 170637,
 'eval_totals': [170637, 156851, 143065, 129279]}

In [ ]:
trainer.evaluate(max_length=max_target_length)

***** Running Evaluation *****
  Num examples = 13786
  Batch size = 64


In [25]:
trainer.evaluate(max_length=max_target_length)

***** Running Evaluation *****
  Num examples = 13786
  Batch size = 64


Trainer is attempting to log a value of "[168009, 152282, 137119, 122447]" of type <class 'list'> for key "eval/counts" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "[170660, 156874, 143088, 129302]" of type <class 'list'> for key "eval/totals" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "[98.44661900855502, 97.07281002588064, 95.82844123895785, 94.69845787381479]" of type <class 'list'> for key "eval/precisions" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.


{'epoch': 1.0,
 'eval_bp': 1.0,
 'eval_counts': [168009, 152282, 137119, 122447],
 'eval_loss': 0.02427506446838379,
 'eval_precisions': [98.44661900855502,
  97.07281002588064,
  95.82844123895785,
  94.69845787381479],
 'eval_ref_len': 170441,
 'eval_runtime': 562.7222,
 'eval_samples_per_second': 24.499,
 'eval_score': 96.50147121893254,
 'eval_steps_per_second': 0.384,
 'eval_sys_len': 170660,
 'eval_totals': [170660, 156874, 143088, 129302]}

In [ ]:
from transformers import pipeline

# Replace this with your own checkpoint
model_checkpoint = "huggingface-course/marian-finetuned-kde4-en-to-fr"
translator = pipeline("translation", model=model_checkpoint)
translator("Default to expanded threads")

In [12]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 124074
    })
    validation: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 13786
    })
})

In [14]:
vld_preds = trainer.predict(tokenized_datasets["validation"])

***** Running Prediction *****
  Num examples = 13786
  Batch size = 64


In [26]:
!pip install wandb

import wandb
wandb.login()

     |████████████████████████████████| 1.7 MB 5.5 MB/s eta 0:00:01
     |████████████████████████████████| 144 kB 36.4 MB/s 
     |████████████████████████████████| 180 kB 46.9 MB/s 
     |████████████████████████████████| 63 kB 1.6 MB/s 
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8806 sha256=eef36672c0b2539bc6187ba5e26d8957492b7a724521fb7748ff4b294ee59d69
  Stored in directory: /root/.cache/pip/wheels/3e/31/09/fa59cef12cdcfecc627b3d24273699f390e71828921b2cbba2
Successfully built pathtools


<IPython.core.display.Javascript object>

wandb: You can find your API key in your browser here: https://wandb.ai/authorize


wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [20]:
vld_preds

PredictionOutput(predictions=array([[59513,    14,     6, ..., 59513, 59513, 59513],
       [59513,     8,  1387, ..., 59513, 59513, 59513],
       [59513,     8, 12502, ..., 59513, 59513, 59513],
       ...,
       [59513,    76,  1146, ..., 59513, 59513, 59513],
       [59513,     8, 40402, ..., 59513, 59513, 59513],
       [59513,   113,  4427, ..., 59513, 59513, 59513]]), label_ids=array([[   14,     6,    18, ..., 59513, 59513, 59513],
       [    8,  1387,    51, ..., 59513, 59513, 59513],
       [    8, 12502,    51, ..., 59513, 59513, 59513],
       ...,
       [   76,  1146,   157, ..., 59513, 59513, 59513],
       [    8, 40402,    43, ..., 59513, 59513, 59513],
       [  113,  4427, 18542, ..., 59513, 59513, 59513]]), metrics={'test_loss': 0.022329561412334442, 'test_score': 96.71274245906265, 'test_counts': [168302, 152687, 137628, 123032], 'test_totals': [170813, 157027, 143241, 129455], 'test_precisions': [98.52997137220235, 97.23614410260656, 96.0814291997403, 95.0384303

### Couple of fine tuning experiments

#### First, I tried to change optimiser to "Adafactor", which yielded 96.5 BLEU score validation set. (Refer second trainer.evaluate() ) Second, I tried to change learning rate to 0.01 , but I was not able to see score , due to COLAB limitations on GPU training.For reference , I added both arguments config.

#### Comparative study:- Here, as you can see, baseline model and transformer model uses different split, so BLEU scores are not directly comparable. One can try preparing dataset in such a manner that test set remains constant across experiments. But tentatively, 96% BLEU score seem to be better than 87% (still not a perfect statement due to different test set)

#### Advance finetuning:- We can try changing other parameters like batch size, defining a learning rate scheduler etc. Advance strategy would be adding adapters on top of pretrained model (freeze its weights). 

#### Referene for this:- https://github.com/Adapter-Hub/adapter-transformers

#### Here, one can follow below process to identify where to add adapters:-
1. Freeze certain layer and see evaluation loss. (no adapters so far)
2. Then, check if eval loss is decreasing. If not, then that layer's weights do not need finetuning.
3. Follow such steps for important layers like embedding, norm, attention layer etc.